### Resize Image

In [1]:
import { Bitmap, Jimp } from '@jimp/core';
const jimp = require("jimp") as typeof import('jimp');

var imageData : Bitmap;

//resize image to 640x640
jimp.read('./sample-images/car-with-licenseplate.jpeg').then((image) => {
  imageData = image.resize(640, 640).bitmap;
  image.resize(640,640).write('./sample-images/car-with-licenseplate-resized-tiny.jpeg');
  console.log("Image resized");
}).catch(err => {
  console.error(err);
});

Promise { <pending> }


Image resized


#### Define Function

In [2]:
//import { Tensor, InferenceSession } from "onnxruntime-web";
// import ort from "onnxruntime-web";
const ort = require('onnxruntime-node') as typeof import('onnxruntime-node'); // onnyruntime-web
const { Tensor } = ort as typeof import('onnxruntime-node');

interface ImageMetadata {
    batch: number;
    channels: number;
    width: number;
    height: number;
}

function imageDataToTensor(data, imageMetaData:ImageMetadata): any {
    // 1a. Extract the R, G, and B channels from the data to form a 3D int array
    const [R, G, B] = new Array([], [], []);
    for (let i = 0; i < data.length; i += 4) {
      R.push(data[i]);
      G.push(data[i + 1]);
      B.push(data[i + 2]);
      // 2. skip data[i + 3] thus filtering out the alpha channel
    }
    console.log('channels extracted');
    ///console.log(R);
    //console.log(G);
    //console.log(B);
    // 1b. concatenate RGB ~= transpose [224, 224, 3] -> [3, 224, 224]
    const transposedData = R.concat(G).concat(B);
    console.log('data transpose complete');

    // 3. convert to float32
    let i, l = transposedData.length; // length, we need this for the loop
    const float32Data = new Float32Array(imageMetaData.batch * imageMetaData.channels * imageMetaData.width * imageMetaData.height); // create the Float32Array for output
    for (i = 0; i < l; i++) {
      float32Data[i] = transposedData[i] / 255.0; // convert to float (pixel value range maybe? 255)
    }
    console.log('data converted to float32');
  
    const inputTensor = new Tensor("float32", float32Data, [imageMetaData.batch, imageMetaData.channels, imageMetaData.width, imageMetaData.height]);
    console.log('inputTensor created');
    // inputTensor.data.set(float32Data);
    console.log('inputTensor data set');
    return inputTensor;
  }

console.log('function defined');

function defined


In [3]:
const { InferenceSession } = ort as typeof import('onnxruntime-node');





async function runModel(session, preprocessedData): Promise<[any, number]> {
    const start = new Date();
    try {
    
     
      const feeds: Record<string,any> = {};
      feeds[session.create().inputNames[0]] = preprocessedData;
      const outputData = await session.run(feeds);
      const end = new Date();
      const inferenceTime = (end.getTime() - start.getTime());
      const output = outputData[session.outputNames[0]];
      return [output, inferenceTime];
      
    } catch (e) {
      console.error(e);
      throw new Error();
    }
  }

In [4]:

import fs  from 'fs';
import { Bitmap, Jimp } from '@jimp/core';
import { InferenceSession } from 'onnxruntime-web';
if (typeof imageData === 'undefined' || imageData === null || typeof imageData.data === 'undefined') {
    throw new Error('imageData is undefined');
}

const imageMetadata = {
    width: (imageData as Bitmap).width,
    height: (imageData as Bitmap).height,
    channels: 3,
    batch: 1
};

console.log('imageMetadata', JSON.stringify(imageMetadata));

const tensor = imageDataToTensor((imageData as Bitmap).data, imageMetadata);
console.log('tensor created');
// create an inference session, using WebGL backend. (default is 'wasm') 
//const session = await ort.InferenceSession.create('./model/squeezenet1_1.onnx', { executionProviders: ['wasm'] }); 
const fileBuffer =  fs.readFileSync('./models/lpr-8n.onnx');
//convert to unit8array
const u8:Uint8Array = new Uint8Array(fileBuffer);
console.log('u8 created');
const session = await InferenceSession.create(u8); //webgl
console.log('session created');

/*
let output;
runModel(session, tensor).then((results) => {   
    output = results;
    console.log('output', results);
}).catch((err) => {
    console.log('err', err);
}  );
*/

//const output = (await runModel(session, tensor))[0];

    const feeds: Record<string,any> = {};
    feeds[session.inputNames[0]] = tensor;
const rawSessionResults = await session.run(feeds);
const output = rawSessionResults[session.outputNames[0]];
console.log('output', output);



imageMetadata {"width":640,"height":640,"channels":3,"batch":1}
channels extracted
data transpose complete
data converted to float32
inputTensor created
inputTensor data set
tensor created
u8 created
session created
output h {
  dims: [ 1, 5, 8400 ],
  type: 'float32',
  data: Float32Array(42000) [
    11.964459419250488,  11.40317153930664, 11.772724151611328,
    20.090736389160156,  41.95893096923828,  59.14415740966797,
     53.87537384033203, 48.173118591308594, 45.515830993652344,
      45.3691520690918, 57.343292236328125,  68.81871032714844,
    115.52296447753906, 115.28607177734375, 124.54100036621094,
    126.21919250488281,  127.9874267578125, 137.27052307128906,
    142.67034912109375, 146.69973754882812,  149.1882781982422,
    159.96739196777344, 181.38278198242188,  196.6282958984375,
    198.09735107421875,  198.4817352294922, 201.08047485351562,
    201.35208129882812,  202.1505584716797, 207.64581298828125,
    214.54132080078125,   221.981201171875,   229.6520996093

In [5]:
//import cv from "@techstark/opencv-js";
const cv = require("@techstark/opencv-js") as typeof import('@techstark/opencv-js');

function processResults(results, xRatio, yRatio){
    const boxes = [];
    //console.log("results.dims:",results.dims);
    // looping through output
    for (let idx = 0; idx < results.dims[1]; idx++) {
      const data = results.data.slice(idx * results.dims[2], (idx + 1) * results.dims[2]); // get rows
      //console.log("data:",data);
      const box = data.slice(0, 4);
      const scores = data.slice(4); // classes probability scores
      const score = Math.max(...scores); // maximum probability scores
      const label = scores.indexOf(score); // class id of maximum probability scores
  
      
      const [x, y, w, h] = [
        (box[0] - 0.5 * box[2]) * xRatio, // upscale left
        (box[1] - 0.5 * box[3]) * yRatio, // upscale top
        box[2] * xRatio, // upscale width
        box[3] * yRatio, // upscale height
      ]; // keep boxes in maxSize range
      
      /*
      const [x, y, w, h] = [
        box[0] ,// upscale left
        box[1] , // upscale top
        box[2] , // upscale width
        box[3] , // upscale height
      ]; // keep boxes in maxSize range
      */
  
      boxes.push({
        label: label,
        probability: score,
        bounding: [x, y, w, h], // upscale box
      }); // update boxes to draw later
      return boxes;
    }

}
const xRatio = 50;
const yRatio = 50;
const boxes = processResults(output, xRatio, yRatio);
console.log(boxes);

[
  {
    label: 6315,
    probability: 636.5421142578125,
    bounding: [
      303.9048671722412,
      67.89016723632812,
      588.6362075805664,
      1004.5368194580078
    ]
  }
]


In [6]:
const { Canvas, createCanvas, Image, ImageData, loadImage } = require('canvas') as typeof import('canvas');
const { JSDOM } = require('jsdom') as typeof import('jsdom');


/**
 * Preprocessing image
 * @param {HTMLImageElement} source image source
 * @param {Number} modelWidth model input width
 * @param {Number} modelHeight model input height
 * @return preprocessed image and configs
 */
const preprocessing = (source, modelWidth, modelHeight) => {
    const mat = cv.imread(source); // read from img tag
    //const mat = cv.asImageData
    const matC3 = new cv.Mat(mat.rows, mat.cols, cv.CV_8UC3); // new image matrix
    cv.cvtColor(mat, matC3, cv.COLOR_RGBA2BGR); // RGBA to BGR
  
    // padding image to [n x n] dim
    const maxSize = Math.max(matC3.rows, matC3.cols); // get max size from width and height
    const xPad = maxSize - matC3.cols, // set xPadding
      xRatio = maxSize / matC3.cols; // set xRatio
    const yPad = maxSize - matC3.rows, // set yPadding
      yRatio = maxSize / matC3.rows; // set yRatio
    const matPad = new cv.Mat(); // new mat for padded image
    cv.copyMakeBorder(matC3, matPad, 0, yPad, 0, xPad, cv.BORDER_CONSTANT); // padding black
  
    const input = cv.blobFromImage(
      matPad,
      1 / 255.0, // normalize
      new cv.Size(modelWidth, modelHeight), // resize to model input size
      new cv.Scalar(0, 0, 0),
      true, // swapRB
      false // crop
    ); // preprocessing image matrix
  
    // release mat opencv
    mat.delete();
    matC3.delete();
    matPad.delete();
  
    return [input, xRatio, yRatio];
  };




// Using jsdom and node-canvas we define some global variables to emulate HTML DOM.
// Although a complete emulation can be archived, here we only define those globals used
// by cv.imread() and cv.imshow().
function installDOM() {
  const dom = new JSDOM();
  global.document = dom.window.document;
  // The rest enables DOM image and canvas and is provided by node-canvas
  global.Image = Image;
  global.HTMLCanvasElement = Canvas;
  global.ImageData = ImageData;
  global.HTMLImageElement = Image;
 }


 try {
  installDOM();
  const image = await loadImage('./sample-images/car-with-licenseplate-resized.jpeg')
  const preprocessResult = preprocessing(image, 640, 640);
  console.log("preprocessResult: ", preprocessResult);
} catch (error) {
  console.log("error: ", error);
}
  

preprocessResult:  [ Mat {}, 1, 1 ]


In [9]:
const labels = ["license plate"];

function renderBoxes  (canvas, boxes) {
    const ctx = canvas.getContext("2d");
    ctx.clearRect(0, 0, ctx.canvas.width, ctx.canvas.height); // clean canvas
  
    const colors = new Colors();
  
    // font configs
    const font = `${Math.max(
      Math.round(Math.max(ctx.canvas.width, ctx.canvas.height) / 40),
      14
    )}px Arial`;
    ctx.font = font;
    ctx.textBaseline = "top";
  
    boxes.forEach((box) => {
      const klass = labels[box.label];
      const color = colors.get(box.label);
      const score = (box.probability * 100).toFixed(1);
      const [x1, y1, width, height] = box.bounding;
  
      // draw box.
      ctx.fillStyle = Colors.hexToRgba(color, 0.2);
      ctx.fillRect(x1, y1, width, height);
      // draw border box
      ctx.strokeStyle = color;
      ctx.lineWidth = Math.max(Math.min(ctx.canvas.width, ctx.canvas.height) / 200, 2.5);
      ctx.strokeRect(x1, y1, width, height);
  
      // draw the label background.
      ctx.fillStyle = color;
      const textWidth = ctx.measureText(klass + " - " + score + "%").width;
      const textHeight = parseInt(font, 10); // base 10
      const yText = y1 - (textHeight + ctx.lineWidth);
      ctx.fillRect(
        x1 - 1,
        yText < 0 ? 0 : yText,
        textWidth + ctx.lineWidth,
        textHeight + ctx.lineWidth
      );
  
      // Draw labels
      ctx.fillStyle = "#ffffff";
      ctx.fillText(klass + " - " + score + "%", x1 - 1, yText < 0 ? 1 : yText + 1);
    });
  };
  
  class Colors {
    public readonly palette: string[];
    public readonly n: number;
    // ultralytics color palette https://ultralytics.com/
    constructor() {
      this.palette = [
        "#FF3838",
        "#FF9D97",
        "#FF701F",
        "#FFB21D",
        "#CFD231",
        "#48F90A",
        "#92CC17",
        "#3DDB86",
        "#1A9334",
        "#00D4BB",
        "#2C99A8",
        "#00C2FF",
        "#344593",
        "#6473FF",
        "#0018EC",
        "#8438FF",
        "#520085",
        "#CB38FF",
        "#FF95C8",
        "#FF37C7",
      ];
      this.n = this.palette.length;
    }
  
    get = (i) => this.palette[Math.floor(i) % this.n];
  
    static hexToRgba = (hex, alpha) => {
      var result = /^#?([a-f\d]{2})([a-f\d]{2})([a-f\d]{2})$/i.exec(hex);
      return result
        ? `rgba(${[parseInt(result[1], 16), parseInt(result[2], 16), parseInt(result[3], 16)].join(
            ", "
          )}, ${alpha})`
        : null;
    };
  }

try {
  const canvas = createCanvas();
  renderBoxes(canvas, boxes);
  fs.writeFileSync('./sample-images/output.jpg', canvas.toBuffer('image/jpeg'));
  console.log('done');
}catch(err) {
  console.log(err);
}


92:18 - Expected 2-3 arguments, but got 0.


UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (/Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/dist/converter.js:111:19)
    at Object.scheduleInvalidateResolutionsOfFailedLookupLocations (/Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:122719:55)
    at scheduleInvalidateResolutionOfFailedLookupLocation (/Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:121553:22)
    at /Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:121617:11
    at /Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5810:11
    at /Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5560:101
    a

: 